In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
filepath_home= 'D:/Dacon_Psychological_disposition/'
filepath_out = 'C:/임시/Dacon_Psychological_disposition/' 
train = pd.read_csv(filepath_home+'train.csv',index_col=0)
test = pd.read_csv(filepath_home+'test_x.csv',index_col=0)

In [3]:
# 분할 지점 저장해두고 데이터 통합 후 전처리
split_point = train.shape[0]
data = pd.concat([train.drop('voted',axis=1),test],ignore_index=True)

__결측 : 121 개__

In [4]:
data.married.value_counts()

1    39478
2    12528
3     4788
0      121
Name: married, dtype: int64

__양적 변수 중 그나마 연관 : education__ 

In [7]:
data.corr(method='spearman')['married'].sort_values(ascending=False)[:15] 

married       1.000000
education     0.278764
tp02          0.182348
wr_11         0.179079
QaE           0.166365
tp08          0.163558
wr_03         0.150832
QgE           0.134741
QsE           0.131361
QfE           0.125900
familysize    0.124464
QrE           0.116973
QnE           0.113559
QtE           0.107309
QoE           0.105881
Name: married, dtype: float64

__질적 변수 중 연관 : age_group -> 확실히 10s ,20s 는 1로 봐도 무방해보임__

In [13]:
data[data.married!=0].groupby(['age_group'])['married'].value_counts().unstack()

married,1,2,3
age_group,,,
+70s,18,182,106
10s,17641,93,31
20s,15386,1895,358
30s,4420,4099,1225
40s,1366,3420,1503
50s,506,1952,1118
60s,141,887,447


__education 만 활용하면 최빈값 적용시 모두 1이 나오므로 age_group 도 같이 그룹핑 해서 최빈값 적용하자__

In [26]:
data[(data.married!=0)&(data.education!=0)].groupby(['education'])['married'].value_counts(normalize=True).unstack()

married,1,2,3
education,,,
1,0.953907,0.025572,0.020521
2,0.782555,0.142490,0.074955
3,0.625717,0.275604,0.098678
4,0.482132,0.401470,0.116398


__같은 연령대임에도 교육정도에 따라 최빈값이 차이남 ==> 최빈값 활용하기 적절해보임__

In [27]:
data[(data.married!=0)&(data.education!=0)].groupby(['age_group','education'])['married'].value_counts(normalize=True).unstack()

married                     1         2         3
age_group education                              
+70s      1          0.181818  0.454545  0.363636
          2          0.068966  0.436782  0.494253
          3          0.044944  0.640449  0.314607
          4          0.053571  0.687500  0.258929
10s       1          0.995608  0.002635  0.001757
          2          0.992437  0.006205  0.001357
          3          0.987977  0.008487  0.003536
          4          0.980198  0.009901  0.009901
20s       1          0.856061  0.098485  0.045455
          2          0.872259  0.097153  0.030589
          3          0.884013  0.100207  0.015780
          4          0.843844  0.141742  0.014414
30s       1          0.552632  0.263158  0.184211
          2          0.478224  0.359296  0.162479
          3          0.455848  0.427431  0.116721
          4          0.423466  0.474368  0.102166
40s       1          0.326316  0.357895  0.315789
          2          0.228395  0.482099  0.289506
          3          0.222769  0.537443  0.239788
          4          0.195759  0.614464  0.189777
50s       1          0.153846  0.353846  0.492308
          2          0.144144  0.490490  0.365365
          3          0.158387  0.539957  0.301656
          4          0.113960  0.612536  0.273504
60s       1          0.111111  0.500000  0.388889
          2          0.052381  0.619048  0.328571
          3          0.110680  0.590291  0.299029
          4          0.112705  0.598361  0.288934

__최빈값 테이블 생성 : fill_table__

In [31]:
fill_table = data[(data.married!=0)&(data.education!=0)].groupby(['age_group','education'])['married'].apply(lambda x : x.value_counts().index[0])
fill_table

age_group  education
+70s       1            2
           2            3
           3            2
           4            2
10s        1            1
           2            1
           3            1
           4            1
20s        1            1
           2            1
           3            1
           4            1
30s        1            1
           2            1
           3            1
           4            2
40s        1            2
           2            2
           3            2
           4            2
50s        1            3
           2            2
           3            2
           4            2
60s        1            2
           2            2
           3            2
           4            2
Name: married, dtype: int64

__결측값에 대하여 연령대 , 교육 수준을 참고하여 채움__

In [38]:
def fill_married_NA(row):
    if row.education == 0 :
        if row.age_group =='10s':
            fill_value = fill_table[(row.age_group,2)]
        else:
            fill_value = fill_table[(row.age_group,3)]
    else:
        fill_value = fill_table[(row.age_group,row.education)]
    
    row.married=fill_value
    return row

data[data.married==0] = data[data.married==0].apply(fill_married_NA , axis=1)


__처리 결과__

In [39]:
data.married.value_counts()

1    39565
2    12562
3     4788
Name: married, dtype: int64